In [1]:
# Loading the required Libraries

import numpy as np
import glob
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word
stop = stopwords.words('english')
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import re

Using TensorFlow backend.


In [2]:
# Reading file 

rawData = pd.read_csv('train_tweets.txt',sep='\t',names=('Id','tweet'))

rawData.shape

(328195, 2)

In [3]:
#Basic Feature Extraction

#Average Length 

def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))


def featureExtraction(rawData):
#Number of Words

    rawData['word_count'] = rawData['tweet'].apply(lambda x: len(str(x).split(" ")))
    rawData[['tweet','word_count']].head()

#Number of characters

    rawData['char_count'] = rawData['tweet'].str.len() ## this also includes spaces
    rawData[['tweet','char_count']].head()




  #  rawData['avg_word'] = rawData['tweet'].apply(lambda x: avg_word(x))
  #  rawData[['tweet','avg_word']].head()

#Number of StopWords



    rawData['stopwords'] = rawData['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
    rawData[['tweet','stopwords']].head()

#Number of Special Charaters 

    rawData['hastags'] = rawData['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
    rawData[['tweet','hastags']].head()

#Numbe rof Numerics 

    rawData['numerics'] = rawData['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
    rawData[['tweet','numerics']].head()

# Number of Upper Case 

    rawData['upper'] = rawData['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
    rawData[['tweet','upper']].head()
    
    return rawData
    
#processedData = featureExtraction(rawData)    
#print(processedData)

In [4]:

def tokenize(text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = nltk.word_tokenize(text)
    filtered_sentence = [] 
    for w in word_tokens: 
        #if w not in stop_words: 
        filtered_sentence.append(w) 
    stems = []
    for item in filtered_sentence:
        stems.append(PorterStemmer().stem(item))
    return ' '.join(word for word in stems)

def preProcessing(rawData):

#Lower case

    rawData['tweet'] = rawData['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    rawData['tweet'].head()

#Punctuation removal

    rawData['tweet'] = rawData['tweet'].str.replace('[^\w\s]','')
    rawData['tweet'].head()

#Stopwords removal

    rawData['tweet'] = rawData['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    rawData['tweet'].head()

#Frequent words removal


    freq = pd.Series(' '.join(rawData['tweet']).split()).value_counts()[:10]
    
    freq = list(freq.index)
    rawData['tweet'] = rawData['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
    rawData['tweet'].head()

#Rare words removal


    freq = pd.Series(' '.join(rawData['tweet']).split()).value_counts()[-10:]

#Spelling correction


    freq = list(freq.index)
    rawData['tweet'] = rawData['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
    rawData['tweet'].head()

#Tokenization

    #rawData['tweet'][:5].apply(lambda x: str(TextBlob(x).correct()))

    #WordList=[]
    #TextBlob(rawData['tweet'][1]).words
    #WordList=['thanks', 'lyft', 'credit', 'cant', 'use', 'cause', 'dont', 'offer', 'wheelchair', 'vans', 'pdx', 'disapointed', 'getthanked']
    rawData['tweet'] = rawData['tweet'].apply(lambda x: tokenize(x))
#Stemming


    st = PorterStemmer()
    rawData['tweet'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

#Lemmatization


    rawData['tweet'] = rawData['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    rawData['tweet'].head()
    
    return rawData;

#processedData  = preProcessing(processedData)

#print(processedData)

In [5]:
#SPLitting Data
num_dataset = rawData

#print(dataset)
#names = ['authorID','NumLink','hastags','spCount','word_count','char_count','isCapitalize','RT_count']


X = num_dataset
y = num_dataset.iloc[:,0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=1000)

encoder = LabelEncoder()


encoder.fit(rawData["Id"])
encoded_Y = encoder.transform(y_train)
encoded_Y.shape


X_train



,Id,tweet
114734,6177,MarketWatch: China's October CPI lower than ex...
66115,671,the melting temperature of an irish man is 32 ...
243897,9615,"@handle Yes, it's very good."
315882,4139,@handle I could totally see @handle giving it ...
307246,3074,"@handle What is this ""podcast maker"" you speak..."
143034,8758,"The Latest News About Law Of Attraction, From ..."
298438,8954,Phewww I didn't get twitter'ed last night lmfao
230664,4833,"@handle yep, that is the link, beautiful boat...."
231310,8414,One of the best buildups to the my favorite zo...
220903,5399,@handle oh jeez -- #embarrassing....unless tha...


In [6]:
encoder.fit(rawData["Id"])
encoded_Y = encoder.transform(y_train)
encoded_Y.shape

(262556,)

In [7]:
x_train_extractedData = featureExtraction(X_train)
x_train_processedData = preProcessing(x_train_extractedData)

C:\Users\Waqar\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Waqar\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Waqar\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [8]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer 
#vectorizer = CountVectorizer(analyzer = "word",\ tokenizer = None, \ preprocessor = None,\stop_words = None,   \ max_features = 5000) 

vectorizer = CountVectorizer()

vectorizer.fit(x_train_processedData['tweet'].values)


Creating the bag of words...



CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

tfidf.fit(x_train_processedData['tweet'].values)



TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [10]:
x_train_matrix_BOW = vectorizer.transform(x_train_processedData['tweet'].values)

x_train_matrix_Tfidf = tfidf.transform(x_train_processedData['tweet'].values)

In [11]:
from sklearn.decomposition import TruncatedSVD

svd_Tfidf = TruncatedSVD(n_components=200)
svd_Tfidf.fit(x_train_matrix_Tfidf)


TruncatedSVD(algorithm='randomized', n_components=200, n_iter=5,
             random_state=None, tol=0.0)

In [12]:

from sklearn.decomposition import TruncatedSVD

svd_bow = TruncatedSVD(n_components=200)
svd_bow.fit(x_train_matrix_BOW)




TruncatedSVD(algorithm='randomized', n_components=200, n_iter=5,
             random_state=None, tol=0.0)

In [13]:
x_trainT_matrix_Tfidf = svd_Tfidf.transform(x_train_matrix_Tfidf)

In [14]:
x_trainT_matrix_BOW = svd_bow.transform(x_train_matrix_BOW)

In [15]:
x_trainT_matrix_Tfidf.shape

(262556, 200)

In [16]:
x_trainT_matrix_BOW.shape

(262556, 200)

In [17]:
data_bow = pd.DataFrame(x_trainT_matrix_BOW)
data_TFID = pd.DataFrame(x_trainT_matrix_Tfidf)
data_TFID.shape

(262556, 200)

In [18]:
featureDat = pd.concat([data_bow, data_TFID], axis=1)
x_train_extractedData.shape


(262556, 8)

In [19]:
featureDat.shape

(262556, 400)

In [20]:
#featureData = pd.concat([x_train_extractedData.drop(columns=["tweet","Id"]), featureDat], axis=1)
#featureData.shape
featureData= featureDat
featureData.shape

(262556, 400)

In [21]:
# Validation Data 

x_test_extractedData = featureExtraction(X_test)
x_test_processedData = preProcessing(x_test_extractedData)


C:\Users\Waqar\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Waqar\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Waqar\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [22]:
 x_test_matrix_BOW= vectorizer.transform(x_test_processedData['tweet'].values)

In [23]:
x_test_matrix_tfidf= tfidf.transform(x_test_processedData['tweet'].values)

In [24]:
x_testT_matrix_Tfidf = svd_Tfidf.transform(x_test_matrix_tfidf)
x_testT_matrix_BOW = svd_bow.transform(x_test_matrix_BOW)




In [25]:
data_bow_val = pd.DataFrame(x_testT_matrix_BOW)
data_TFID_val = pd.DataFrame(x_testT_matrix_Tfidf)

In [26]:
featureDat_val = pd.concat([data_bow_val, data_TFID_val], axis=1)
#featureData_val = pd.concat([x_test_extractedData.drop(columns=["tweet","Id"]), featureDat_val], axis=1)
featureData_val = featureDat_val
featureData_val.shape
featureData_val

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1.882701e-01,-5.439227e-02,4.719603e-02,-1.740133e-02,2.144696e-03,6.251927e-02,9.566993e-02,-7.549085e-02,-7.609839e-02,2.977343e-01,...,0.006671,0.008138,-3.014846e-03,-1.651744e-02,0.014115,0.012621,0.011660,-1.074082e-03,0.022754,-0.008135
1,2.500368e-02,-1.266626e-02,5.338297e-03,1.682856e-03,-1.963450e-03,1.235232e-02,1.394371e-03,-3.023791e-03,-1.595371e-02,2.686400e-04,...,-0.013372,0.012642,2.958669e-03,-7.551845e-03,0.004307,0.005072,-0.013440,-9.410997e-03,-0.024231,0.001013
2,8.109503e-03,-3.133081e-03,-5.583316e-04,5.176724e-04,-1.324243e-03,2.515306e-04,2.428874e-03,7.420935e-04,-1.206062e-03,1.646857e-03,...,-0.000256,-0.000374,-2.541159e-04,-5.253905e-04,-0.000006,-0.000451,-0.000740,1.058451e-04,-0.001611,-0.000283
3,7.953641e-02,-3.414776e-02,3.024797e-02,5.445881e-03,-6.479880e-03,3.270364e-02,4.833722e-03,-2.184314e-02,-1.545645e-02,9.153051e-03,...,-0.002282,0.014829,-1.388555e-02,-2.558417e-02,0.018247,0.001568,0.007147,-2.748732e-03,-0.010677,0.014612
4,2.886361e-01,-8.551399e-02,1.315622e-01,-5.289504e-02,5.054337e-02,-3.007206e-02,3.623853e-02,3.686935e-03,2.443576e-02,2.647707e-02,...,-0.035751,-0.032355,9.796540e-03,-1.092597e-02,-0.005453,-0.031668,-0.019083,2.898960e-02,-0.005935,0.006599
5,3.751486e-01,-2.153033e-01,2.589146e-02,1.458551e-01,-1.426716e-01,1.402606e-01,1.831746e-01,-3.336387e-01,-1.605425e-02,6.012336e-01,...,-0.080554,0.027839,5.646690e-02,-7.415878e-02,0.044465,-0.100355,-0.045624,2.349694e-02,-0.000240,0.011123
6,3.210686e-02,-1.239415e-02,4.837490e-03,-2.225516e-03,-8.037012e-04,5.796763e-03,7.268051e-03,1.420147e-05,-6.290993e-03,4.722636e-03,...,0.003858,-0.003550,2.931411e-03,-1.760270e-03,-0.002263,0.006564,-0.010111,-1.765186e-02,-0.006459,-0.015048
7,3.076932e-01,-1.362416e-01,1.034135e-01,-6.547591e-02,1.371114e-01,-5.260899e-03,-6.595176e-02,-3.892848e-02,1.313786e-02,2.821794e-02,...,-0.013003,-0.005384,9.944980e-04,1.206310e-03,-0.016116,-0.005837,0.005002,-4.991397e-04,0.014530,-0.001680
8,1.422907e-01,-1.123726e-01,5.735894e-02,6.060891e-02,4.720572e-02,1.975318e-01,-1.155881e-01,1.442822e-01,-5.977550e-01,-2.070506e-01,...,0.006208,-0.000425,1.269050e-02,4.110982e-03,0.005020,0.010345,-0.011847,1.509190e-02,0.004342,-0.002268
9,1.438651e-01,-3.891074e-02,3.893660e-02,-7.125250e-03,-4.014382e-02,5.374264e-02,4.674625e-02,-2.955786e-02,-2.937930e-02,4.239005e-02,...,0.003864,-0.012153,-6.247394e-04,1.294312e-02,-0.018407,-0.006214,-0.008409,1.413655e-02,0.019070,0.008943


In [27]:

encoded_test_Y = encoder.transform(y_test)
encoded_test_Y


array([4834, 3083, 6701, ..., 3625,  615, 6687], dtype=int64)

In [28]:
input_dim = featureData.shape[1]
input_dim
featureData.shape

(262556, 400)

In [ ]:
# ANN Libraries 


from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
# Model of ANN

input_dim = featureData.shape[1]
model = Sequential()
model.add(Dense(32, input_dim=input_dim, activation='relu')) # input dimension = dimension of festure vector
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(9293, activation='softmax')) # output layer = no. of classes
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


batch_size = 32
epochs = 30
history = model.fit(featureData,encoded_Y,batch_size=batch_size, epochs=epochs,verbose=1,validation_data=(featureData_val,encoded_test_Y))

Train on 262556 samples, validate on 65639 samples
Epoch 1/30
262556/262556 [==============================] - 96s 366us/step - loss: 8.8849 - acc: 0.0026 - val_loss: 8.7040 - val_acc: 0.0056
Epoch 2/30
262556/262556 [==============================] - 95s 362us/step - loss: 8.4948 - acc: 0.0093 - val_loss: 8.4987 - val_acc: 0.0117
Epoch 3/30
262556/262556 [==============================] - 95s 363us/step - loss: 8.2493 - acc: 0.0147 - val_loss: 8.4300 - val_acc: 0.0163
Epoch 4/30
262556/262556 [==============================] - 95s 362us/step - loss: 8.1097 - acc: 0.0186 - val_loss: 8.3992 - val_acc: 0.0180
Epoch 5/30
262556/262556 [==============================] - 95s 363us/step - loss: 8.0182 - acc: 0.0215 - val_loss: 8.3954 - val_acc: 0.0199
Epoch 6/30
262556/262556 [==============================] - 95s 362us/step - loss: 7.9513 - acc: 0.0236 - val_loss: 8.3879 - val_acc: 0.0212
Epoch 7/30
262556/262556 [==============================] - 95s 361us/step - loss: 7.8993 - acc: 0.0255

In [30]:
#Model Predections
def readFile(path):
    with open(path, encoding="utf8") as f:
        lineList = f.readlines()
    return lineList

testFile = readFile('test_tweets_unlabeled.txt')
testData = pd.DataFrame(testFile,columns = ['tweet'])


testData.shape
testData



,tweet
0,Some people say that rappers don’t have feelin...
1,Do you know how to tweet on a Blackberry 8830?...
2,"""Yoga is the cessation of mind."" -Patanjali\n"
3,"@handle Well, with my millions of dollars, a f..."
4,Cambria hotels free guide http://hotels.izigot...
5,May the force of Jesus be with you http://ff.i...
6,YEAH! It's finally Monday!\n
7,Martin Laird won in Las Vegas last week with a...
8,Joe's Crab Shack Fundraiser benefitting the Sa...
9,"i hate my self-portrait painting, I don't know..."


In [31]:
x_predict_extractedData = featureExtraction(testData)
x_predict_processedData = preProcessing(x_predict_extractedData)

In [32]:
x_predict_matrix_BOW= vectorizer.transform(x_predict_processedData['tweet'].values)

x_predict_matrix_tfidf= tfidf.transform(x_predict_processedData['tweet'].values)

In [33]:
x_predictT_matrix_Tfidf = svd_Tfidf.transform(x_predict_matrix_tfidf)
x_predictT_matrix_BOW = svd_bow.transform(x_predict_matrix_BOW)


In [34]:
pdata_bow_val = pd.DataFrame(x_predictT_matrix_BOW)
pdata_TFID_val = pd.DataFrame(x_predictT_matrix_Tfidf)

In [35]:
predictDat_val = pd.concat([pdata_bow_val, pdata_TFID_val], axis=1)
featureData_val = pd.concat([x_test_extractedData.drop(columns=["tweet","Id"]), featureDat_val], axis=1)
pfeatureData_val =predictDat_val
pfeatureData_val.shape
pfeatureData_val

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.682912,-0.364786,0.390192,-0.231780,0.519338,-0.270641,-0.100898,-0.061225,0.064958,-0.057535,...,0.003068,-0.006064,-0.002288,0.008920,0.002210,0.006666,-0.000479,0.000885,0.000369,-0.000768
1,0.409127,-0.389574,-0.299244,0.628175,0.576745,-0.603535,-0.200262,-0.146139,0.089517,-0.140022,...,-0.041439,-0.018960,-0.025140,0.021340,-0.090624,0.014583,-0.013044,0.014971,-0.007203,0.017396
2,0.016805,-0.006161,0.007365,-0.001821,0.002993,-0.001345,0.001160,-0.003522,0.000850,0.004581,...,-0.005703,0.019045,0.000803,0.010264,-0.002575,-0.011444,0.002473,-0.002587,-0.017660,-0.010518
3,0.261402,-0.138562,0.167408,-0.009239,0.042958,0.464103,-0.587870,0.261831,0.436410,0.021288,...,0.000969,0.002025,0.004647,-0.005408,0.002948,0.002546,-0.003127,0.008806,0.001678,-0.006398
4,0.145704,-0.093106,0.082730,0.036456,-0.012249,0.233202,-0.149934,0.036346,-0.170520,-0.078364,...,-0.028358,-0.007021,-0.008235,-0.015523,-0.018821,-0.017796,0.018404,-0.028547,0.040543,0.021309
5,0.029831,-0.012583,0.007643,-0.001494,0.000837,0.007082,0.004352,-0.008204,0.002387,-0.002922,...,0.045973,-0.003694,0.015848,-0.016849,-0.029122,0.029460,0.009183,0.048147,-0.015585,-0.015733
6,0.059741,-0.017005,0.011676,0.003381,-0.010444,0.004813,0.034649,0.017916,0.000673,0.008761,...,0.066500,-0.033221,0.021852,0.021906,-0.037521,-0.041006,0.040091,-0.004838,-0.031057,-0.032780
7,0.191785,-0.073226,0.042014,0.033823,-0.094718,0.074276,0.118711,-0.084945,-0.010208,0.110246,...,0.010553,-0.009113,-0.017799,0.017055,0.018959,-0.005518,0.001239,-0.007246,-0.012046,-0.012101
8,0.076247,-0.008383,0.017910,0.006876,-0.016153,0.024738,0.043478,-0.024923,0.011584,0.006685,...,-0.010061,0.006599,-0.001416,-0.000433,-0.012875,0.001111,-0.001912,-0.002596,0.008734,0.001660
9,0.378621,-0.234280,0.410853,-0.267949,0.775906,-0.695660,-0.185304,-0.310889,0.075522,-0.141881,...,-0.013713,-0.009842,-0.005489,-0.004620,0.007875,-0.003906,0.010055,0.001048,-0.003799,-0.006004


In [36]:
X_ptestdata = pfeatureData_val.iloc[:, 0:].values

y_ptestdata = model.predict_classes(X_ptestdata)
y_poutput = encoder.inverse_transform(y_ptestdata)

#y_poutput
result = []
for i in range(0,len(y_poutput)):
    result.append(y_poutput[i])
print(len(result))
resDF = pd.DataFrame(result)
resDF.index += 1

resDF

#y_poutput

35437


,0
1,6455
2,5506
3,4948
4,4948
5,8297
6,8224
7,2306
8,1650
9,7781
10,9592


In [37]:
resDF.to_csv("predictionFilev4.csv")